In [ ]:
import cartopy.crs as ccrs
import dask
import matplotlib.pyplot as plt
import metpy
from netCDF4 import Dataset
import numpy as np
import pandas as pd
import scipy as sp
import xarray

from postprocess_utils import *

# Model Variables
In the following cell you can set the values of the variables relevant to the model. The details of each variable are included in the README. In most cases it is only necessary to set values for the standard variables. Note that any variable included in the model should be given the same value in the postprocess. For example, if the model used zw = 42 and kmax = 11, you should use zw = 42 and kmax = 11 below.

In [ ]:
# Set postprocess parameters.

# Standard Variables
zw = 42
kmax = 11
expname = 'Control'
DataSetname = 'vvel'
Dataname = 'v'
dayst = 30

# Advanced Variables
imax = None
jmax = None
custom_path = None
custom_kmax = None

In [ ]:
# Set Dependent Variables

# Set value of kmax if custom_kmax is used.
if not(custom_kmax is None):
    kmax = custom_kmax
    print("Using custom value for kmax:", kmax)
# Otherwise check value for kmax.
elif kmax != 11 and kmax != 26:
    raise Exception(
        "Unexpected value for kmax."
        " Use custom_kmax and note that other values are implementable,"
        " but the user must modify subs1_utils.py routine bscst."
        " If unclear email bkirtman@miami.edu for clarification.")

# Check value for zw.
# Afterwards, set jmax and imax values based on the value given to zw.
# If a value is already given for one of the listed variables,
# use that instead.
match zw:
    case 42:
        jmax = 64 if (jmax is None) else jmax
        imax = 128 if (imax is None) else imax
    case 63:
        jmax = 96 if (jmax is None) else jmax
        imax = 192 if (imax is None) else imax
    case 124:
        jmax = 188 if (jmax is None) else jmax
        imax = 376 if (imax is None) else imax
    case _:
        if (jmax is None) or (imax is None):
            raise Exception(
                "Unexpected value for zw. Other values are implementable,"
                " but the user must specify values for jmax and imax"
                " in the advanced variables section.")
print("zw =", zw,
      "\nkmax =", kmax,
      "\njmax =", jmax,
      "\nimax =", imax,
      "\ndayst =", dayst)

In [ ]:
# Set datapath.
datapath = get_model_data_path(custom_path, expname)

# Set stamp for file names
stamp = 'days_1-' + str(dayst)
print("stamp =", stamp)

In [ ]:
fps = datapath + 'lnps_1*.nc' # Always need surface pressure.
dps = xarray.open_mfdataset(fps, decode_times=True, parallel=True)

fdata = datapath + DataSetname + '_1*.nc'
ddata = xarray.open_mfdataset(fdata, decode_times=True, parallel=True)

In [ ]:
fps

In [ ]:
dps

In [ ]:
ddata

In [ ]:
# Create Data Array for Control Pressure level Data geopotenial,
# temp, u & v.
lats = ddata['lat'].values
lons = ddata['lon'].values
plev = [1000.0, 900.0, 800.0, 700.0, 600.0, 500.0, 400.0, 300.0, 200.0, 100.0,
        20.0]
plev_r = np.zeros(11)
for k in range(11):
    plev_r[k] = (plev[k]) * 100.0 # mb to Pa

# "11" here corresponds to standard pressure levels not to model levels.
dout = np.zeros((dayst, 11, jmax, imax))
pressure = np.zeros((kmax, jmax, imax))
siglevs = ddata['lev']
for k in range (dayst):
    vv = ddata[Dataname][k, :, :, :]
    ps = dps.lnps[k, :, :]
    surfp = (np.exp(ps)) * 1000.0 * 100.0 # in Pa
    for kk in range(kmax):
        pressure[kk, :, :] = surfp[:, :] * siglevs[kk]
    vv = vv.compute()
    ps = ps.compute()
    dout[k] = metpy.interpolate.log_interpolate_1d(
        plev_r, pressure, vv, axis=0)

times = ddata['time']
dData = xarray.Dataset(
    {Dataname: (['time', 'lev', 'lat', 'lon'], dout)},
    coords={'time': times, 'lev': plev, 'lat': lats, 'lon': lons})

In [ ]:
dData

In [ ]:
dData.to_netcdf(datapath+DataSetname + '_Pressure_' + stamp + '.nc')

In [ ]:
dData[Dataname][dayst-1, 0, :, :].plot()

In [ ]:
dData[Dataname][dayst-1, 7, :, :].plot()

In [ ]:
dData[Dataname][dayst-1, :, 45, 64].plot()